## Importing the goodies 

In [5]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 
from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
from mss.linux import MSS as mss
import mss.tools
import cv2
import pandas as pd 
import os

In [8]:

def countdown(sec):
    for i in range(sec):
        print(sec - i)
        time.sleep(1)

def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]


def screen_record_efficient(top = 225, left = 0, width = 800, height = 260, resize_width = 80, resize_height = 25):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (resize_width, resize_height))
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    sct.close()
    return img



def flex(np_img, name = "foo"):
    cv2.imshow(name, np_img)
    
    
def canny(gray):
    im = cv2.Canny(gray,50,500,apertureSize = 3)
    return im

def fill_voids(img_np):
    kernel = np.ones((2,2),np.uint8)
    img = cv2.morphologyEx(img_np, cv2.MORPH_CLOSE, kernel, iterations = 1)
    return img


    
def segment_road_on_np(image, lower_gray = [0, 0, 1], higher_gray = [179, 50, 100]):
    img = (image).astype(np.uint8)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_gray = np.array(lower_gray, np.uint8)
    upper_gray = np.array(higher_gray, np.uint8)
    mask_gray = cv2.inRange(hsv, lower_gray, upper_gray)
    img_res = cv2.bitwise_and(img, img, mask = mask_gray)
    return mask_gray



In [13]:


def collect_ensemble_data(flex_mode = False, segment_road = True, main_folder = "ensemble_frames"):
    print ("running...")
    print ("drag mouse to the corner of the screen to terminate -")
    
    instances = []
    while True:
        
        
        upper_raw = screen_record_efficient(top = 225 + 200 + 45,   ## main + lower + remove sky
                                      width = 800, 
                                      height = 142, 
                                      resize_width = 128, 
                                      resize_height = 64)  ## FPV TRACK VIEW  
    
        
        lower_raw = screen_record_efficient(top = 225 ,   ## main + lower + remove sky
                                              width = 800, 
                                              height = 142, 
                                              resize_width = 64, 
                                              resize_height = 64)  ## FPV TRACK VIEW  
        
         
        mouse =find_mouse(x = True)
        steer = 400 - mouse
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

        if segment_road == True:
            upper_seg = canny(upper_raw)


        if flex_mode == True:
            flex(upper_seg , name = "upper")
            flex(lower_raw, name = "lower")


        time_val = str(int(time.time()*100))[2:]
        name_upper = "upper_" + str(steer)+"_" + time_val + ".jpg"
        name_lower = "lower_" + str(steer)+"_" + time_val + ".jpg"


        save_name_upper = main_folder + "/" + "upper/" + name_upper
        save_name_lower = main_folder + "/" + "lower/" + name_lower
        
        cv2.imwrite(save_name_upper, upper_seg)
        cv2.imwrite(save_name_lower, lower_raw)
        
        instances.append([save_name_upper, save_name_lower, steer])
    return instances[:-10]

In [14]:
countdown(2)
instances = collect_ensemble_data(flex_mode = True, segment_road = True)  

2
1
running...
drag mouse to the corner of the screen to terminate -


In [14]:
df= pd.DataFrame(instances, columns = ["upper", "lower", "steer"])
df.to_csv("training_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7855 entries, 0 to 7854
Data columns (total 3 columns):
upper    7855 non-null object
lower    7855 non-null object
steer    7855 non-null int64
dtypes: int64(1), object(2)
memory usage: 184.2+ KB
